In [1]:
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Flatten
from tensorflow.keras.layers import Embedding
from keras.models import Sequential
from tensorflow.keras import layers, models, losses, optimizers

from keras.layers import Dense, Conv1D, MaxPooling1D, GlobalMaxPooling1D,LSTM,SpatialDropout1D
from keras.utils import pad_sequences

from keras.utils.np_utils import to_categorical
import pandas as pd
import numpy as np
import re

import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split

from gensim.models import Word2Vec
stopwords = stopwords.words('english')

In [2]:
df = pd.read_csv('data/labeled_lyrics_w_genres.csv')
df.shape

(145250, 7)

In [6]:
df_dropped = df[(df['genre'] == 'No_genre') | (df['genre'] == 'Non-Music')|(df['genre'] == 'Rap')  ].index
df.drop(df_dropped, inplace=True, axis='index')



In [7]:
df.shape

(96326, 7)

In [8]:
lemmatizer = WordNetLemmatizer()
# 1. function that makes all text lowercase.
def make_lowercase(test_string):
    return test_string.lower()

# 2. function that removes all punctuation. 
def remove_punc(test_string):
    test_string = re.sub(r'[^\w\s]', '', test_string)
    return test_string

# 3. function that removes all stopwords.
def remove_stopwords(test_string):
    # Break the sentence down into a list of words
    words = word_tokenize(test_string)
    
    # Make a list to append valid words into
    valid_words = []
    
    # Loop through all the words
    for word in words:
        
        # Check if word is not in stopwords. Stopwords was imported from nltk.corpus
        if word not in stopwords:
            
            # If word not in stopwords, append to our valid_words
            valid_words.append(word)

    # Join the list of words together into a string
    a_string = ' '.join(valid_words)

    return a_string

# 4. function to break words into their stem words
def stem_words(a_string):
    # Initalize our Stemmer
    porter = PorterStemmer()
    
    # Break the sentence down into a list of words
    words = word_tokenize(a_string)
    
    # Make a list to append valid words into
    valid_words = []

    # Loop through all the words
    for word in words:
        # Stem the word
        stemmed_word = porter.stem(word) #from nltk.stem import PorterStemmer
        
        # Append stemmed word to our valid_words
        valid_words.append(stemmed_word)
        
    # Join the list of words together into a string
    a_string = ' '.join(valid_words)

    return a_string 
def lem_words(a_string):
    # Initalize our Stemmer
    lemmatizer = WordNetLemmatizer()
    
    # Break the sentence down into a list of words
    words = word_tokenize(a_string)
    
    # Make a list to append valid words into
    valid_words = []

    # Loop through all the words
    for word in words:
        # Stem the word
        lemmed_word = lemmatizer.lemmatize(word) #from nltk.stem import PorterStemmer
        
        # Append stemmed word to our valid_words
        valid_words.append(lemmed_word)
        
    # Join the list of words together into a string
    a_string = ' '.join(valid_words)

    return a_string 

In [9]:
test = "crying and cry"
lem_words(test)

'cry and cry'

In [10]:
# Pipeline

def text_processing_pipeline(a_string):
    a_string = make_lowercase(a_string)
    a_string = remove_punc(a_string)
    a_string = remove_stopwords(a_string)
    a_string = lem_words(a_string)
    return a_string



In [11]:
df['clean_lyrics'] = df.seq.apply(text_processing_pipeline)

df.head()

,Unnamed: 0.1,Unnamed: 0,artist,seq,song,label,genre,clean_lyrics
0,0,0,Elijah Blake,"No, no\r\nI ain't ever trapped out the bando\r...",Everyday,0.626,R&B,aint ever trapped bando oh lord dont get wrong...
1,1,1,Elijah Blake,"The drinks go down and smoke goes up, I feel m...",Live Till We Die,0.630,Pop,drink go smoke go feel got let go care get los...
2,2,2,Elijah Blake,She don't live on planet Earth no more\r\nShe ...,The Otherside,0.240,R&B,dont live planet earth found love venus thats ...
3,3,3,Elijah Blake,"Trippin' off that Grigio, mobbin', lights low\...",Pinot,0.536,R&B,trippin grigio mobbin light low trippin grigio...
4,4,4,Elijah Blake,"I see a midnight panther, so gallant and so br...",Shadows & Diamonds,0.371,R&B,see midnight panther gallant brave found found...


In [13]:
df.genre.value_counts()

Pop        57357
Rock       26756
Country     7440
R&B         4773
Name: genre, dtype: int64

In [14]:
def getNum(gen):
    if gen == 'Pop':
        return 3
    elif gen == 'Rock':
        return 2
    elif gen == 'Country':
        return 1
    else :
        return 0

In [15]:
df['genre'] = df.genre.apply(lambda x: getNum(x))

In [16]:
df = df.sample(frac=1)

X = df['clean_lyrics']

y = df['genre'].values

y = to_categorical( y )
X_text = X

In [17]:
# Limiting our tokenizers vocab size
max_words = 10000
 
    
# create the tokenizer
tokenizer = Tokenizer(num_words=max_words)


# Fit the tokenizer
tokenizer.fit_on_texts(X)


# Create the sequences for each sentence, basically turning each word into its index position
sequences = tokenizer.texts_to_sequences(X)


index_word = tokenizer.index_word


# # Limiting our sequencer to only include 500 words
max_length = 300


# # Convert the sequences to all be the same length of 500
X = pad_sequences(sequences, maxlen=max_length, padding='post')
print(X.shape)

(96326, 300)


In [18]:
# This creates the Neural Network
model = Sequential() 

# This embedding layer basically will automatically create the word2vec vectors based on your text data.
model.add( Embedding(max_words, 32, input_length=max_length) ) 





model.add(LSTM(50,dropout =0.2))


model.add(Dense(4, activation='softmax'))
optimizer = optimizers.Adam(lr=0.003)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) 

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 32)           320000    
                                                                 
 lstm (LSTM)                 (None, 50)                16600     
                                                                 
 dense (Dense)               (None, 4)                 204       
                                                                 
Total params: 336,804
Trainable params: 336,804
Non-trainable params: 0
_________________________________________________________________


c:\users\stephen williams\downloads\nlp-extra-credit-main\env\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [19]:
hist = model.fit(X, y, 
                 validation_split=0.2, 
                 epochs=30, batch_size=20)

Epoch 1/30
3853/3853 [==============================] - 137s 35ms/step - loss: 1.0054 - accuracy: 0.5957 - val_loss: 0.9883 - val_accuracy: 0.5935
Epoch 2/30
3853/3853 [==============================] - 140s 36ms/step - loss: 0.9655 - accuracy: 0.5957 - val_loss: 0.9550 - val_accuracy: 0.5934
Epoch 3/30
3739/3853 [============================>.] - ETA: 3s - loss: 0.9315 - accuracy: 0.5994

KeyboardInterrupt: 

In [69]:
# This creates the Neural Network
model = Sequential() 

# This embedding layer basically will automatically create the word2vec vectors based on your text data.
model.add( Embedding(max_words, 32, input_length=max_length) ) 




model.add(LSTM(50, return_sequences=True,dropout =0.2))
model.add(LSTM(50,dropout =0.2))
model.add(Dense(32))

model.add(Dense(5, activation='softmax'))
optimizer = optimizers.Adam(lr=0.003)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) 

model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, 200, 32)           160000    
                                                                 
 lstm_32 (LSTM)              (None, 200, 50)           16600     
                                                                 
 lstm_33 (LSTM)              (None, 50)                20200     
                                                                 
 dense_20 (Dense)            (None, 32)                1632      
                                                                 
 dense_21 (Dense)            (None, 5)                 165       
                                                                 
Total params: 198,597
Trainable params: 198,597
Non-trainable params: 0
_________________________________________________________________


c:\users\stephen williams\downloads\nlp-extra-credit-main\env\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [70]:
hist = model.fit(X, y, 
                 validation_split=0.2, 
                 epochs=30, batch_size=20)

Epoch 1/30
4092/4092 [==============================] - 407s 99ms/step - loss: 1.0480 - accuracy: 0.5908 - val_loss: 1.0150 - val_accuracy: 0.5947
Epoch 2/30
4092/4092 [==============================] - 408s 100ms/step - loss: 0.9627 - accuracy: 0.6026 - val_loss: 0.9434 - val_accuracy: 0.6063
Epoch 3/30
4092/4092 [==============================] - 391s 96ms/step - loss: 0.9070 - accuracy: 0.6166 - val_loss: 0.9204 - val_accuracy: 0.6118
Epoch 4/30
4092/4092 [==============================] - 387s 95ms/step - loss: 0.8725 - accuracy: 0.6270 - val_loss: 0.9360 - val_accuracy: 0.6066
Epoch 5/30
4092/4092 [==============================] - 388s 95ms/step - loss: 0.8452 - accuracy: 0.6365 - val_loss: 0.9219 - val_accuracy: 0.6096
Epoch 6/30
4092/4092 [==============================] - 392s 96ms/step - loss: 0.8190 - accuracy: 0.6489 - val_loss: 0.9333 - val_accuracy: 0.6005
Epoch 7/30
4092/4092 [==============================] - 395s 97ms/step - loss: 0.7987 - accuracy: 0.6575 - val_loss: 

KeyboardInterrupt: 

In [86]:
# This creates the Neural Network
model = Sequential() 

# This embedding layer basically will automatically create the word2vec vectors based on your text data.
model.add( Embedding(max_words, 100, input_length=max_length) ) 

model.add(SpatialDropout1D(0.2))


model.add(LSTM(100,dropout =0.2,recurrent_dropout=0.2))
model.add(Dense(5, activation='softmax'))

optimizer = optimizers.Adam(lr=0.003)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) 

model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_21 (Embedding)    (None, 250, 100)          5000000   
                                                                 
 spatial_dropout1d_4 (Spatia  (None, 250, 100)         0         
 lDropout1D)                                                     
                                                                 
 lstm_41 (LSTM)              (None, 100)               80400     
                                                                 
 dense_30 (Dense)            (None, 5)                 505       
                                                                 
Total params: 5,080,905
Trainable params: 5,080,905
Non-trainable params: 0
_________________________________________________________________


In [87]:
hist = model.fit(X, y, 
                 validation_split=0.2, 
                 epochs=50, batch_size=64)

Epoch 1/50
1279/1279 [==============================] - 1258s 982ms/step - loss: 1.0546 - accuracy: 0.5894 - val_loss: 1.0212 - val_accuracy: 0.5962
Epoch 2/50
1279/1279 [==============================] - 1283s 1s/step - loss: 0.9914 - accuracy: 0.5993 - val_loss: 0.9679 - val_accuracy: 0.6060
Epoch 3/50
1279/1279 [==============================] - 1242s 971ms/step - loss: 0.9188 - accuracy: 0.6179 - val_loss: 0.9396 - val_accuracy: 0.6100
Epoch 4/50
1279/1279 [==============================] - 1245s 973ms/step - loss: 0.8462 - accuracy: 0.6488 - val_loss: 0.9511 - val_accuracy: 0.6029
Epoch 5/50
1279/1279 [==============================] - 1249s 977ms/step - loss: 0.7692 - accuracy: 0.6866 - val_loss: 0.9887 - val_accuracy: 0.5949
Epoch 6/50
1279/1279 [==============================] - 1253s 980ms/step - loss: 0.6911 - accuracy: 0.7261 - val_loss: 1.0218 - val_accuracy: 0.5923
Epoch 7/50
1279/1279 [==============================] - 1251s 978ms/step - loss: 0.6220 - accuracy: 0.7566 - 

KeyboardInterrupt: 

In [17]:
# This creates the Neural Network
model = Sequential() 

# This embedding layer basically will automatically create the word2vec vectors based on your text data.
model.add( Embedding(max_words, 32, input_length=max_length) ) 



model.add(LSTM(50,return_sequences = True,dropout =0.2,recurrent_dropout=0.2))
model.add(LSTM(50,return_sequences = True,dropout =0.2,recurrent_dropout=0.2))
model.add(LSTM(50,return_sequences = True,dropout =0.2,recurrent_dropout=0.2))
model.add(LSTM(50,return_sequences = True,dropout =0.2,recurrent_dropout=0.2))
model.add(LSTM(50,return_sequences = True,dropout =0.2,recurrent_dropout=0.2))
model.add(LSTM(50,dropout =0.2,recurrent_dropout=0.2))
model.add(Dense(5, activation='softmax'))

optimizer = optimizers.Adam(lr=0.003)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) 

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 300, 32)           320000    
                                                                 
 lstm_10 (LSTM)              (None, 300, 50)           16600     
                                                                 
 lstm_11 (LSTM)              (None, 300, 50)           20200     
                                                                 
 lstm_12 (LSTM)              (None, 300, 50)           20200     
                                                                 
 lstm_13 (LSTM)              (None, 300, 50)           20200     
                                                                 
 lstm_14 (LSTM)              (None, 300, 50)           20200     
                                                                 
 lstm_15 (LSTM)              (None, 50)               

In [ ]:
hist = model.fit(X, y, 
                 validation_split=0.2, 
                 epochs=50, batch_size=64)

Epoch 1/50
1279/1279 [==============================] - 2005s 2s/step - loss: 1.1775 - accuracy: 0.5611 - val_loss: 1.1830 - val_accuracy: 0.5584
Epoch 2/50
1279/1279 [==============================] - 1789s 1s/step - loss: 1.1785 - accuracy: 0.5600 - val_loss: 1.1848 - val_accuracy: 0.5583
Epoch 3/50
1279/1279 [==============================] - 1757s 1s/step - loss: 1.1744 - accuracy: 0.5614 - val_loss: 1.1839 - val_accuracy: 0.5583
Epoch 4/50
1279/1279 [==============================] - 1780s 1s/step - loss: 1.1741 - accuracy: 0.5614 - val_loss: 1.1840 - val_accuracy: 0.5583
Epoch 5/50
1279/1279 [==============================] - 1767s 1s/step - loss: 1.1741 - accuracy: 0.5614 - val_loss: 1.1826 - val_accuracy: 0.5583
Epoch 6/50
1279/1279 [==============================] - 1759s 1s/step - loss: 1.1740 - accuracy: 0.5614 - val_loss: 1.1833 - val_accuracy: 0.5583
Epoch 7/50
1279/1279 [==============================] - 1762s 1s/step - loss: 1.1737 - accuracy: 0.5614 - val_loss: 1.1849 -